In [1]:
# magic incantation to make the notebook wider
from IPython.core.display import HTML
HTML("<style>.container { width:90% !important; }</style>")

In [2]:
import numpy as np
import pylab as pl
import os, sys
# %matplotlib inline

import new_functions as fn
fn = reload(fn)

/Users/mfisherlevine/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [3]:
RUN = 10

if RUN == 10:
    beam_mon_file = '/Users/mfisherlevine/Desktop/desy/RUN10.txt'
#     timepix_data_dir = '/Users/mfisherlevine/Desktop/desy/Data/Run10/'
    timepix_data_dir = '/Users/mfisherlevine/Desktop/desy/test/'

    MCP_file = ''

In [4]:
tp_datafiles = [timepix_data_dir + _ for _ in os.listdir(timepix_data_dir)]
print 'Found %s datafiles for run %s'%(len(tp_datafiles), RUN)

Found 1 datafiles for run 10


In [ ]:
# GLITCH_THRESHOLD = 25000

# XMIN = 0
# XMAX = 255
# YMIN = 0
# YMAX = 255

# TMIN = 0
# TMAX = 11810

# TCUT = 257

# # data_path = '/Users/mfisherlevine/Desktop/desy/Data/Run10/'
# data_path = '/Users/mfisherlevine/Desktop/desy/test/'

# # data_path = 'C:\\Users\\Merlin\\Desktop\\2015_nov\\Thr_sweep\\'
# prefix = '?*'

# # savepath =  'C:\\Users\\Merlin\\Desktop\\DESY_analysis\\Run14\\'
# # savepath =  'C:\\Dropbox\\new_plots\\Run14\\Radial\\'
# savepath =  '/Users/mfisherlevine/Desktop/temp/'

In [ ]:
# LoadFiles(maxfiles = 9999999)

In [ ]:
# TOF_TMIN = 0
# TOF_TMAX = 11810

# TOF_TMIN = 8100
# TOF_TMAX = 8550

# # TOF_TMIN = 8280
# # TOF_TMAX = 8340

# fig = pl.figure(figsize=[16,8])

# ax_raw, ax_cent = ShowTOF(flip=True, centroided=True, fig=fig)
# fig.tight_layout()
# # filename = savepath + 'Run10_12k_files_peak_zoom2.pdf'

# fig.savefig(filename)

In [ ]:
print len(data['/Users/mfisherlevine/Desktop/desy/test/a_2015_12_17_18_11_00_202.txt']['ts'])
print sorted(data['/Users/mfisherlevine/Desktop/desy/test/a_2015_12_17_18_11_00_202.txt']['ts'])

In [ ]:
print sc.use_CoM_as_centroid
print sc.npix_per_cluster_cut

In [10]:
import SmartCentroider
SmartCentroider = reload(SmartCentroider)

sc = SmartCentroider.SmartCentroider(tp_datafiles)
sc.skiplines = 1
sc.ToF_noise_threshold = 0
sc.n_tof_files = 500
sc.TMAX = 1e9
sc.TMIN = -1e9
sc.inc_diagnoal_joins = True
sc.npix_per_cluster_cut=(0,1e9)
sc.use_CoM_as_centroid=False

sc.DEBUG=-1
sc.bands=[(0,11810)]
sc.run()
data = sc.ret

Smart centroided 0 of 1 files...
Finished smart centroiding 1 files...
Found 52 timecodes after centroiding


In [ ]:
sc.MakeSampleTOF()
sc.CalculateBands()

print 'Smoothed, noise suppressed and truncated ToF spectrum for sample files'
sc.ShowTOF(sc.sample_TOF_smoothed)
sc.ShowBands()
sc.PrintBandsForEditing()

In [8]:
import new_functions as fn
import scipy.ndimage as ndimage
from scipy.ndimage.measurements import center_of_mass, maximum_position
import pylab as plt

In [7]:
# sc.skiplines = 1
# sc.ToF_noise_threshold = 0
# sc.n_tof_files = 500
TMAX = 1e9
TMIN = -1e9
# sc.inc_diagnoal_joins = True
# sc.npix_per_cluster_cut=(0,1e9)

In [9]:
test = FindClusters(tp_datafiles)

In [11]:
print len(test[tp_datafiles[0]]['ts'])
print sorted(test[tp_datafiles[0]]['ts'])

NameError: name 'test' is not defined

In [5]:
img = None
def FindClusters(filenames):
    global img
    bands = [(-999,1e9)]
    struct_el=[[1,1,1],[1,1,1],[1,1,1]] # for including diagonal connections as well
    # struct_el=[[0,1,0],[1,1,1],[0,1,0]] # for vertical/horizontal connections only
    ret = {}
    #     xs, ys, ts = [], [], [] #return lists
    for filenum, filename in enumerate(filenames):
        fileID = filename
        ret[fileID]={'xs':[],'ys':[],'ts':[],'npixs':[]}

        img = fn.TimepixFileToImage(filename, skiplines=1, t_min=TMIN, t_max=TMAX)

        segmentation, segments = ndimage.label(img, struct_el) # find clusters

        seg_sum = 0
        for bandnum,(tmin, tmax) in enumerate(bands): # Process each band in turn
            band_img = img.copy() # make a copy
            # TODO: Check that the correct edge is being included/excluded so you're not taking one band edge
            # twice and the other not at all!
            band_img[band_img > tmax] = 0 # threshold the new image
            band_img[band_img <= tmin] = 0

            # find clusters
            segmentation, segments = ndimage.label(band_img, struct_el) 
            seg_sum += segments
#             if self.DEBUG>=3: print 'Found %s segs in band %s (%s - %s)'%(segments, bandnum, tmin, tmax);sys.stdout.flush()

            # Get centroids from clusters using specified method:
            if True: # use center of mass weighting
                # TODO: check whether cluster 0 is included in CoMs
                # TODO: check order of these, i.e. if x,y,t will be coherent
                CoMs = center_of_mass(band_img, segmentation, [_ for _ in xrange(1, segments+1)]) #TODO: replace with range(1,segments+1) ??
                # NB do not replace with enumerate, you need to know the actual number for each cluster - think about it.
                for clust_num in xrange(segments): # cluster 0 = background so skip it
                    clust_pix_index = np.where(segmentation==clust_num) # find pixels associated with cluster
                    ret[fileID]['ts'].append(np.max(img[clust_pix_index]))
                    ret[fileID]['xs'].append(CoMs[clust_num][0])
                    ret[fileID]['ys'].append(CoMs[clust_num][1])
                    ret[fileID]['npixs'].append(len(clust_pix_index[0]))

            else: # Take the earliest timecode in the cluster as the centroid
                max_positions = maximum_position(band_img, segmentation, [_ for _ in xrange(1,segments+1)])
                for max_pos in max_positions: # Stepping through together, so def. coherent
                    self.ret[fileID]['ts'].append(img[max_pos]) # TODO XXXX: I think this is wrong, should be same as logic above
                    self.ret[fileID]['xs'].append(max_pos[0]) 
                    self.ret[fileID]['ys'].append(max_pos[1])
                    self.ret[fileID]['npixs'].append('????')

#             if self.DEBUG>=2: print 'CoMs for band %s: %s'%(bandnum, CoMs);sys.stdout.flush()
        #     index = (np.asarray([_[0] for _ in CoMs]),np.asarray([[_[1] for _ in CoMs]]))
        #     codes = img[CoMs]

#         if self.DEBUG>=2: print 'Found %s clusters when using bands'%seg_sum;sys.stdout.flush() 
#     print 'Finished smart centroiding %s files...'%(len(self.filelist));sys.stdout.flush()
    return ret


In [ ]:
plt.imshow(img)

In [ ]:
print img[54,36]

In [38]:
bands = [(-999,1e9)]
struct_el=[[1,1,1],[1,1,1],[1,1,1]] # for including diagonal connections as well
# struct_el=[[0,1,0],[1,1,1],[0,1,0]] # for vertical/horizontal connections only
ret = {}

filename = tp_datafiles[0]

fileID = filename
ret[fileID]={'xs':[],'ys':[],'ts':[],'npixs':[]}

img = fn.TimepixFileToImage(filename, skiplines=1, t_min=TMIN, t_max=TMAX)

# segmentation, segments = ndimage.label(img, struct_el) # find clusters
# segmentation[segmentation!=5]=0
# plt.imshow(segmentation, cmap='gray')

seg_sum = 0
for bandnum,(tmin, tmax) in enumerate(bands): # Process each band in turn
    # Get centroids from clusters using specified method:
    if False: # use center of mass weighting
        for clust_num in xrange(1, segments+1): # cluster 0 = background so skip it
            clust_pix_index = np.where(segmentation==clust_num) # find pixels associated with cluster
            t = np.max(img[clust_pix_index])
            x = CoMs[clust_num-1][0] + 0.5 # +0.5 to evaluate weights at centres of pixels
            y = CoMs[clust_num-1][1] + 0.5
            npix = len(clust_pix_index[0])
            print t,x,y,npix
            ret[fileID]['ts'].append(t)
            ret[fileID]['xs'].append(x)
            ret[fileID]['ys'].append(y)
            ret[fileID]['npixs'].append(npix)

    else: # Take the earliest timecode in the cluster as the centroid
        max_positions = maximum_position(band_img, segmentation, [_ for _ in xrange(1,segments+1)])
#         for max_pos in max_positions: # Stepping through together, so def. coherent
        for clust_num in xrange(1, segments+1): # cluster 0 = background so skip it
            ret[fileID]['ts'].append(img[max_positions[clust_num-1]])
            ret[fileID]['xs'].append(max_positions[clust_num-1][0] +.5) 
            ret[fileID]['ys'].append(max_positions[clust_num-1][1] +.5)
            npix = len(segmentation[segmentation==(clust_num)])
            ret[fileID]['npixs'].append(npix)

In [24]:
a = np.array(([0,0,0,0],
              [0,1,1,0],
              [0,1,1,0],
              [0,0,0,0]))
from scipy import ndimage
ndimage.measurements.center_of_mass(a)

(1.5, 1.5)

In [ ]:
np.dot()

In [12]:
ret = sc.ret

In [13]:
tuples = zip(ret[tp_datafiles[0]]['xs'],
             ret[tp_datafiles[0]]['ys'],
             ret[tp_datafiles[0]]['ts'],
             ret[tp_datafiles[0]]['npixs'])
for x,y,t,npix in sorted(tuples, key=lambda tup: tup[2]):
    print t,x,y, npix

7740.0 54.5 36.5 1
8201.0 120.5 125.5 1
8217.0 130.5 127.5 4
8218.0 116.5 127.5 9
8224.0 80.5 45.5 11
8230.0 101.5 153.5 13
8230.0 183.5 171.5 7
8230.0 188.5 179.5 11
8234.0 126.5 120.5 1
8258.0 131.5 122.5 10
8259.0 136.5 119.5 4
8262.0 121.5 139.5 4
8262.0 122.5 148.5 7
8276.0 102.5 132.5 13
8364.0 139.5 123.5 1
8395.0 91.5 132.5 25
8395.0 107.5 116.5 7
8397.0 70.5 137.5 7
8397.0 126.5 122.5 4
8397.0 144.5 133.5 12
8401.0 138.5 147.5 7
8403.0 184.5 140.5 10
8405.0 107.5 128.5 10
8409.0 59.5 156.5 12
8410.0 137.5 208.5 4
8411.0 99.5 111.5 20
8412.0 124.5 198.5 14
8412.0 125.5 110.5 10
8413.0 137.5 212.5 31
8414.0 158.5 85.5 8
8416.0 141.5 153.5 17
8418.0 143.5 124.5 14
8419.0 52.5 195.5 16
8423.0 74.5 184.5 14
8423.0 98.5 172.5 14
8480.0 134.5 162.5 8
8480.0 175.5 181.5 13
8481.0 79.5 114.5 22
8481.0 139.5 116.5 9
8481.0 147.5 173.5 18
8482.0 134.5 139.5 198
8482.0 161.5 103.5 14
8493.0 45.5 224.5 3
8495.0 48.5 218.5 4
8495.0 103.5 245.5 5
8496.0 39.5 198.5 11
8496.0 51.5 38.5 14
8498

In [37]:
# print len(tuples)
for x,y,t,npix in sorted(tuples, key=lambda tup: tup[2]):
    print t,x,y

7740.0 54.5 36.5
8201.0 120.5 125.5
8217.0 130.0 128.0
8218.0 116.5 128.5
8224.0 80.3181818182 45.0454545455
8230.0 102.269230769 153.423076923
8230.0 184.214285714 171.5
8230.0 188.409090909 180.136363636
8234.0 126.5 120.5
8258.0 131.0 122.3
8259.0 136.0 120.0
8262.0 121.0 140.0
8262.0 122.5 148.214285714
8276.0 101.346153846 132.038461538
8364.0 139.5 123.5
8395.0 92.98 132.06
8395.0 107.785714286 116.5
8397.0 70.7857142857 137.5
8397.0 127.0 123.0
8397.0 144.0 133.0
8401.0 137.785714286 147.5
8403.0 184.5 140.7
8405.0 107.5 128.7
8409.0 59.6666666667 157.5
8410.0 137.0 208.0
8411.0 98.55 111.8
8412.0 124.0 198.785714286
8412.0 124.7 110.5
8413.0 137.370967742 213.629032258
8414.0 159.0 85.5
8416.0 142.5 153.852941176
8418.0 144.285714286 123.285714286
8419.0 53.0 196.5
8423.0 74.2142857143 184.0
8423.0 98.2142857143 172.0
8480.0 134.0 162.5
8480.0 175.884615385 181.884615385
8481.0 80.4090909091 114.590909091
8481.0 139.5 116.5
8481.0 147.222222222 175.666666667
8482.0 130.01010101

In [40]:
# print len(tuples)
for x,y,t,npix in sorted(tuples, key=lambda tup: tup[2]):
    print t,x,y

7740.0 54 36
8201.0 120 125
8217.0 130 127
8218.0 116 127
8224.0 80 45
8230.0 101 153
8230.0 183 171
8230.0 188 179
8234.0 126 120
8258.0 131 122
8259.0 136 119
8262.0 121 139
8262.0 122 148
8276.0 102 132
8364.0 139 123
8395.0 91 132
8395.0 107 116
8397.0 70 137
8397.0 126 122
8397.0 144 133
8401.0 138 147
8403.0 184 140
8405.0 107 128
8409.0 59 156
8410.0 137 208
8411.0 99 111
8412.0 124 198
8412.0 125 110
8413.0 137 212
8414.0 158 85
8416.0 141 153
8418.0 143 124
8419.0 52 195
8423.0 74 184
8423.0 98 172
8480.0 134 162
8480.0 175 181
8481.0 79 114
8481.0 139 116
8481.0 147 173
8482.0 134 139
8482.0 161 103
8493.0 45 224
8495.0 48 218
8495.0 103 245
8496.0 39 198
8496.0 51 38
8498.0 176 137
8501.0 57 235
8502.0 80 98
8507.0 135 71
9525.0 69 243
